In [21]:
# Reading the dataset

import pandas as pd
import os
os.chdir('C:/Users/dell/Downloads/AjayB')
df = pd.read_csv("ccx_row_input_data.csv",encoding='latin-1',nrows=200)
df = df[['profileId', 'programId', 'score', 'name']]

In [22]:
df.shape

(200, 4)

In [23]:
df.head()

,profileId,programId,score,name
0,[ee535a1d-6034-4bd9-a154-b21dd7a8e03c],667969ThirdParty3331353638303536,30.00,L'enlÌ¬vement 2
1,[df82733a-97c1-4002-81fa-fe6933600eb3],667969ThirdParty3332393432343436,30.00,ÌÛ la valdrague
2,[15221437-5617-4bba-a57e-ab7febed831e],667969ThirdParty373732323236,30.00,Taking Chance
3,[c6f96fd7-a955-4fb2-9446-06be4d80e09a],667969ThirdParty3330313932323231,10.64,Rapides et dangereux
4,[ea34f2aa-0baa-49e3-b7a7-a7873b2b8db6],667969ThirdParty3330313932323231,35.48,Rapides et dangereux


In [24]:
data = df.dropna()

In [25]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [26]:
data['profileId'] = data['profileId'].astype("category")
data['profileId'] = le.fit_transform(data['profileId'])

In [27]:
data['programId'] = data['programId'].astype("category")
le_1 = LabelEncoder()
data['programId'] = le_1.fit_transform(data['programId'])

In [28]:
data['name'] = data['name'].astype("category")
le_2 = LabelEncoder()
data['name'] = le_2.fit_transform(data['name'])


In [29]:
data.head()

,profileId,programId,score,name
0,117,24,30.00,25
1,105,31,30.00,45
2,9,42,30.00,35
3,94,21,10.64,28
4,114,21,35.48,28


In [30]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext

sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-10-e8291d723f8a>:6 

In [31]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()



In [32]:
# ppandas dataframe to spark dataframe
dfs = spark.createDataFrame(data)

In [33]:
# train test split , 80 20 
dfs_train, dfs_test = dfs.randomSplit([0.8, 0.2])

In [35]:
from pyspark.ml.recommendation import ALS

RANK = 10
MAX_ITER = 15
REG_PARAM = 0.05

COL_USER = "profileId"
COL_ITEM = "name"
COL_RATING = "score"
COL_PREDICTION = "prediction"

als = ALS(
    maxIter=MAX_ITER, 
    rank=RANK,
    regParam=REG_PARAM, 
    userCol=COL_USER, 
    itemCol=COL_ITEM, 
    ratingCol=COL_RATING, 
    coldStartStrategy="drop"
)

model = als.fit(dfs_train)

In [36]:
dfs_pred = model.transform(dfs_test)


In [37]:
dfs_pred.show()

+---------+---------+-----+----+----------+
|profileId|programId|score|name|prediction|
+---------+---------+-----+----+----------+
|       14|       12| 30.0|  34| 18.648788|
|       66|       12| 30.0|  34| 24.802433|
|      109|       29|100.0|  44| 108.64346|
|       67|       29|100.0|  44|-11.501342|
|       78|       35| 30.0|  13| 2.3072357|
|       90|       25| 30.0|   6| 14.850961|
|       85|       11| 90.0|   7|-15.695039|
|       13|        9|  5.8|  33|  1.316195|
|       49|        9|  5.8|  33|  1.316195|
|       52|       34|100.0|   2| 52.290718|
|        9|       34|100.0|   2|-82.710045|
|       63|       34|100.0|   2| 46.334053|
|      113|       34| 30.0|   2| 11.058773|
|       98|       34|100.0|   2| 20.532871|
|       78|       16|100.0|  36|  14.55468|
|        5|       16|100.0|  36|  44.66456|
|       59|       16| 30.0|  36|  56.11152|
+---------+---------+-----+----+----------+



In [38]:
users = dfs_train.select('profileId').distinct()
items = dfs_train.select('name').distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)

In [39]:
# Remove seen items.
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    dfs_train.alias("train"),
    (dfs_pred['profileId'] == dfs_train['profileId']) & (dfs_pred['name'] == dfs_train['name']),
    how='outer'
)

In [40]:
dfs_pred_exclude_train.show()

+---------+----+----------+---------+---------+-----+----+
|profileId|name|prediction|profileId|programId|score|name|
+---------+----+----------+---------+---------+-----+----+
|       15|  18| 7.5308795|     null|     null| null|null|
|       21|   9|-5.3684297|     null|     null| null|null|
|       22|  25| 16.632195|     null|     null| null|null|
|       25|  24| 1.1157551|     null|     null| null|null|
|       27|   0|  8.114546|     null|     null| null|null|
|       27|  20| 11.365269|     null|     null| null|null|
|       29|   3| 25.464739|     null|     null| null|null|
|       29|  37|-1.1786822|     null|     null| null|null|
|       33|  36| 29.997887|       33|       16| 30.0|  36|
|       34|   6|  8.337781|     null|     null| null|null|
|       35|  30|  6.886712|     null|     null| null|null|
|       61|  34| 47.174744|     null|     null| null|null|
|       68|  21|  5.609314|     null|     null| null|null|
|       82|  10| 32.681553|     null|     null| null|nul

In [41]:
dfs_pred_final = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.score"].isNull()).select('pred.' + 'profileId', 'pred.' + 'name', 'pred.' + "prediction")

dfs_pred_final.take(2)

[Row(profileId=15, name=18, prediction=7.530879497528076),
 Row(profileId=21, name=9, prediction=-5.368429660797119)]

In [42]:
dfs_pred_final.show()

+---------+----+----------+
|profileId|name|prediction|
+---------+----+----------+
|       15|  18| 7.5308795|
|       21|   9|-5.3684297|
|       22|  25| 16.632195|
|       25|  24| 1.1157551|
|       27|   0|  8.114546|
|       27|  20| 11.365269|
|       29|   3| 25.464739|
|       29|  37|-1.1786822|
|       34|   6|  8.337781|
|       35|  30|  6.886712|
|       61|  34| 47.174744|
|       68|  21|  5.609314|
|       82|  10| 32.681553|
|       83|  25| 0.1082592|
|       86|   9|-21.818848|
|       87|  33|  4.647318|
|       92|  10|  8.589676|
|      108|   8|-19.429737|
|      112|  20|  9.365735|
|      112|  39| 18.050344|
+---------+----+----------+
only showing top 20 rows



In [43]:
# Top k for a selected set of users (items)¶

users = dfs_train.select(als.getUserCol()).distinct().limit(3)

dfs_rec_subset = model.recommendForUserSubset(users, 10)

dfs_rec_subset.show(10)

+---------+--------------------+
|profileId|     recommendations|
+---------+--------------------+
|       65|[[21, 29.997046],...|
|       26|[[2, 99.998886], ...|
|       29|[[44, 99.99711], ...|
+---------+--------------------+



In [44]:
dfs_rec = model.recommendForAllUsers(10)

dfs_rec.show(10)

+---------+--------------------+
|profileId|     recommendations|
+---------+--------------------+
|       31|[[44, 108.64346],...|
|       85|[[16, 89.99821], ...|
|       65|[[21, 29.997046],...|
|       78|[[38, 59.99846], ...|
|      108|[[34, 39.693142],...|
|       34|[[44, 62.329063],...|
|      101|[[2, 99.999275], ...|
|      115|[[44, 108.64346],...|
|      126|[[22, 70.752655],...|
|       76|[[44, 65.18608], ...|
+---------+--------------------+
only showing top 10 rows



In [45]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="name",
                                predictionCol="prediction")

rmse = evaluator.evaluate(dfs_pred)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 31.195965162209717


In [47]:
from reco_utils.evaluation.spark_evaluation import SparkRankingEvaluation, SparkRatingEvaluation


In [49]:
#Number of recommended items

K = 10


evaluations = SparkRankingEvaluation(
    dfs_test, 
    dfs_pred_final,
    col_user=COL_USER,
    col_item=COL_ITEM,
    col_rating=COL_RATING,
    col_prediction=COL_PREDICTION,
    k=K
)

print(
    "Precision@k = {}".format(evaluations.precision_at_k()),
    "Recall@k = {}".format(evaluations.recall_at_k()),
    "NDCG@k = {}".format(evaluations.ndcg_at_k()),
    "Mean average precision = {}".format(evaluations.map_at_k()),
    sep="\n"
)

Precision@k = 0.06875
Recall@k = 0.65625
NDCG@k = 0.40143496423697733
Mean average precision = 0.30937499999999996


In [53]:
pandasOutput = dfs_pred_final.toPandas()

In [57]:
pandasOutput.head()

,profileId,name,prediction
0,15,18,7.530879
1,21,9,-5.368430
2,22,25,16.632195
3,25,24,1.115755
4,27,0,8.114546


In [97]:
backup = pandasOutput

In [98]:
backup.profileId = le.inverse_transform(backup.profileId)

ValueError: y contains previously unseen labels: ['[071cbaa4-d762-4b34-a6b3-44b0f3717097]'
 '[09a5d78a-e17d-4e17-b0bb-d8129265c95d]'
 '[0a49485c-a47e-49ba-bcaa-16356472787b]'
 '[0de3eebb-54d0-4c2b-a257-a720d755f739]'
 '[0ebe48fc-5172-46cf-9184-b5de238b2eec]'
 '[12f4a239-d802-4d95-861f-fff133ff67d4]'
 '[140346b1-6e3d-4554-8860-b9be194a9a85]'
 '[148cb703-6388-4d5c-b9e2-db4e39aea0b7]'
 '[15221437-5617-4bba-a57e-ab7febed831e]'
 '[162931c4-59ca-4028-9e10-9c0a6e28823a]'
 '[16bc0f43-a61b-4e9d-8d76-11495c44bb45]'
 '[170b074d-f3bd-422f-9264-9778eecd1534]'
 '[17d0108b-6f05-4464-99a2-c3ea8a1087f4]'
 '[183ab5c2-92c2-4a48-b8fc-032f0ee6f7f1]'
 '[21c34c1d-23c6-4191-ae9a-5c3f91cb2cb9]'
 '[241b1d65-5905-42b2-9104-5c1580991f15]'
 '[28b8b56a-335f-4a21-8254-8e3527c4fb0b]'
 '[2a23e3e9-d943-4f30-a36f-597f5b1f774f]'
 '[2c6bf78c-5e2c-4a57-ba8e-dde8980757b8]'
 '[2e8ebf1e-dd97-492d-8f50-79917a4e3efc]'
 '[31ffcb2f-62c0-4ef5-8d01-33924a772bde]'
 '[3302bd0f-3972-4611-b61b-1ad3b616965b]'
 '[3965253d-511b-4cbe-bdad-

In [88]:
backup.head()

,profileId,name,prediction
0,[17d0108b-6f05-4464-99a2-c3ea8a1087f4],18,7
1,[2a23e3e9-d943-4f30-a36f-597f5b1f774f],9,-5
2,[2c6bf78c-5e2c-4a57-ba8e-dde8980757b8],25,16
3,[31ffcb2f-62c0-4ef5-8d01-33924a772bde],24,1
4,[3965253d-511b-4cbe-bdad-3c34e61c9a1e],0,8


In [110]:
newbackup = backup[(backup.prediction>0) & (backup.prediction<=45)]

In [113]:
newbackup.prediction = le_2.inverse_transform(newbackup.prediction)
newbackup.name = le_2.inverse_transform(newbackup.name)

C:\Users\dell\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [114]:
newbackup.head()

,profileId,name,prediction
0,[17d0108b-6f05-4464-99a2-c3ea8a1087f4],Friday the 13th Part VII: The New Blood,CTV Toronto News at 6
2,[2c6bf78c-5e2c-4a57-ba8e-dde8980757b8],L'enlÌ¬vement 2,Fixer Upper
3,[31ffcb2f-62c0-4ef5-8d01-33924a772bde],Jersey Shore,A Cure for Wellness
4,[3965253d-511b-4cbe-bdad-3c34e61c9a1e],101 S02E01,Carter
5,[3965253d-511b-4cbe-bdad-3c34e61c9a1e],Henchmen,David Lynch: The Art Life


In [116]:
newbackup.to_csv('Output.csv',index = False)